# Using Jurassic-1 Grande on SageMaker through Model Packages

This sample notebook shows you how to deploy Jurassic-1 Grande using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites:
1. Before running this notebook, please make sure you got this notebook from the model catalog on SageMaker AWS Management Console.
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**.
1. This noebook is intended to work with **boto3 v1.25.4** or higher.

## Contents:
1. [Select model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create an input prompt](#B.-Create-an-input-prompt)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Create your own CV Profile!](#D.-Create-your-own-CV-Profile!)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Select to the model package
Confirm that you recieved this notebook from model catalog on SageMaker AWS Management Console.

In [ ]:
# Mapping for Model Packages (initially only us-east-1 and eu-west-1 is supported)
model_package_map = {
    "us-east-1": "arn:aws:sagemaker:us-east-1:865070037744:model-package/j1-grande-dbf0249-66a2031af8f2315babd86234e01824e6"
    "us-east-2": "arn:aws:sagemaker:us-east-2:057799348421:model-package/j1-grande-dbf0249-66a2031af8f2315babd86234e01824e6"
    "us-west-1": "arn:aws:sagemaker:us-west-1:382657785993:model-package/j1-grande-dbf0249-66a2031af8f2315babd86234e01824e6"
    "us-west-2": "arn:aws:sagemaker:us-west-2:594846645681:model-package/j1-grande-dbf0249-66a2031af8f2315babd86234e01824e6"
    "ca-central-1": "arn:aws:sagemaker:ca-central-1:470592106596:model-package/j1-grande-dbf0249-66a2031af8f2315babd86234e01824e6"
    "eu-central-1": "arn:aws:sagemaker:eu-central-1:446921602837:model-package/j1-grande-dbf0249-66a2031af8f2315babd86234e01824e6"
    "eu-west-1": "arn:aws:sagemaker:eu-west-1:985815980388:model-package/j1-grande-dbf0249-66a2031af8f2315babd86234e01824e6"
    "eu-west-2": "arn:aws:sagemaker:eu-west-2:856760150666:model-package/j1-grande-dbf0249-66a2031af8f2315babd86234e01824e6"
    "eu-west-3": "arn:aws:sagemaker:eu-west-3:843114510376:model-package/j1-grande-dbf0249-66a2031af8f2315babd86234e01824e6"
    "eu-north-1": "arn:aws:sagemaker:eu-north-1:136758871317:model-package/j1-grande-dbf0249-66a2031af8f2315babd86234e01824e6"
    "ap-southeast-1": "arn:aws:sagemaker:ap-southeast-1:192199979996:model-package/j1-grande-dbf0249-66a2031af8f2315babd86234e01824e6"
    "ap-southeast-2": "arn:aws:sagemaker:ap-southeast-2:666831318237:model-package/j1-grande-dbf0249-66a2031af8f2315babd86234e01824e6"
    "ap-northeast-2": "arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/j1-grande-dbf0249-66a2031af8f2315babd86234e01824e6"
    "ap-northeast-1": "arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/j1-grande-dbf0249-66a2031af8f2315babd86234e01824e6"
    "ap-south-1": "arn:aws:sagemaker:ap-south-1:077584701553:model-package/j1-grande-dbf0249-66a2031af8f2315babd86234e01824e6"
    "sa-east-1": "arn:aws:sagemaker:sa-east-1:270155090741:model-package/j1-grande-dbf0249-66a2031af8f2315babd86234e01824e6"
}

In [ ]:
import json
from sagemaker import ModelPackage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import sagemaker as sage
import boto3

### Check the version of boto3 - must be v1.25.4 or higher
If you see a lower version number, pick another kernel to run the notebook, with Python 3.8 or above

In [ ]:
boto3.__version__

### Install ai21 python SDK

In [ ]:
! pip install -U ai21-python-client[SM]
import ai21

In [ ]:
region = boto3.Session().region_name
if region not in model_package_map.keys():
    raise ("UNSUPPORTED REGION")

model_package_arn = model_package_map[region]

In [ ]:
role = get_execution_role()
sagemaker_session = sage.Session()

runtime_sm_client = boto3.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).

In [ ]:
model_name = "j1-grande"

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.g5.12xlarge"
)

### A. Create an endpoint

In [ ]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name, 
                         model_data_download_timeout=3600,
                         container_startup_health_check_timeout=600,
                        )

Once endpoint has been created, you would be able to perform real-time inference.

### B. Interact with the model

You can think of Jurassic-1 Grande as a smart auto-completion algorithm: it is very good at latching on to hints and patterns expressed in plain English, and generating text that follows the same patterns.

Here are two words you should know:
- **Prompt** - the input you provide to the model.
- **Completion** - the output text the model generated.

Enter a simple prompt: "To be or", and let the model complete it

In [ ]:
response = ai21.Completion.execute(sm_endpoint="j1-grande",
                                   prompt="To be or",
                                   maxTokens=4,
                                   temperature=0,
                                   numResults=1)

print(response['completions'][0]['data']['text'])

As you can see, the model identifies the beginning of a famous quote, and completes it correctly.

### C. Create a few-shot prompt

The best way to guide the model is to provide several examples of input-output pairs in the prompt. This establishes a pattern for the model to mimic. Then add the input for a query example and let the model complete it with an appropriate generation. 

In this example, we will create a prompt that generates CV profiles. By "CV Profile" we mean a short paragraph that you could find in a resume, highlighting the candidate’s background, skills and ambitions.

To do this, we will build a few-shot prompt comprised of the following:

1. Prefix with 3 examples. Each example contains the relevant inputs (a role and some skills to highlight) and the output (a profile fitting the role and skills). They are separated by "##".

2. The query inputs. An unseen role and set of skills for which we would like the model to output a new CV profile. These should follow the same format of the inputs in the prefix.


First, we collect some example data for the prompt prefix:

In [ ]:
EXAMPLES_DATA = [
    {"role": "Business Manager", 
     "skills": ["Logical mind", "Problem solver", "2 years of experience in management", "Eager to learn"], 
     "profile": "I have a clear, logical mind with a practical approach to problem-solving and a drive to see things through to completion. I have more than 2 years of experience in managing and leading teams across multiple sectors. I am eager to learn, I enjoy overcoming challenges, and I have a genuine interest in Business Management and making organisations successful."},
    {"role": "Chemical Engineer",
     "skills": ["Hard worker and devoted", "Background in: design, plant operations, offshore operations, and process and safety improvements", "Experience in: designing, testing and analysing processes"],
     "profile": "I am a dedicated, hardworking and proactive Chemical Engineer with a strong background in design, plant operations, offshore operations, and process and safety improvements. I have solid work experience in designing, testing and analysing processes to increase the overall efficiency of operations. I am currently looking for an opportunity to utilise my technical skills in a challenging working environment and become a valuable asset to the organisation that I work for."},
    {"role": "IT Professional",
     "skills": ["5 years experience", "Record designing websites, networking and managing databases", "Excellent interpersonal skills", "Looking for a challenge"],
     "profile": "I am a highly competent IT professional with 5 years of relevant industry experience and  a proven track record in designing websites, networking and managing databases. I have strong technical skills as well as excellent interpersonal skills, enabling me to interact with a wide range of clients. I am eager to be challenged in order to grow and further improve my IT skills. My greatest passion is in life is using my technical know-how to benefit other people and organisations."
    }
]

Then, we use the following helper functions to construct the prefix:

In [ ]:
def enumerate_skills(skills):
    return "\n".join(
        f"{i:d}. {s}" for i, s in enumerate(skills, start=1)
    )

def make_single_example(role, skills, profile):   
    example = f"Write a winning CV profile for {role} incorporating the following features:\n"
    example += enumerate_skills(skills)
    example += "\n\n"
    example += "Profile:\n"
    example += profile
    
    return example

SEPARATOR = "\n\n##\n\n"

FEW_SHOT_PREFIX = SEPARATOR.join(
    make_single_example(x["role"], x["skills"], x["profile"]) for x in EXAMPLES_DATA
)

And finally, we create a function to handle query inputs and create the full prompt:

In [ ]:
def create_cv_profile_prompt(role, skills):
    """
    Create a few-shot prompt to generate CV profiles with Jurassic-1 Grande given a role and skills
    The prompt contains a preset sequence of examples followed by the query role and skills
    """
    return FEW_SHOT_PREFIX + SEPARATOR + make_single_example(role, skills, '')  # keep the profile blank and let the model generate

Let's see how this looks for a Sales Executive role with some sample skills

In [ ]:
prompt = create_cv_profile_prompt(
    role="Sales Executive", 
    skills=["Energetic and ambitious", "6 years of selling experience", "Independent", "MBA from Stanford"]
)
print(prompt)

### D. Perform real-time inference

Time to put Jurassic-1 Grande to work!

Let it generate a CV profile for the given input. Since the generation is not deterministic, you can ask for several completions per request and pick the one you prefer.

In [ ]:
response = ai21.Completion.execute(sm_endpoint="j1-grande",
                                   prompt=prompt,
                                   maxTokens=100,
                                   temperature=0.7,
                                   stopSequences=['##'],
                                   numResults=3)

for comp in response['completions']:
    print(comp['data']['text'].strip())
    print("=============")

### E. Create your own CV Profile!

In [ ]:
role = input("Role:")
skills = input("Skills, separated by commas with no spaces: (for example, \'Hard worker,Resourceful\')")
your_prompt = create_cv_profile_prompt(role, skills.split(","))

In [ ]:
response = ai21.Completion.execute(sm_endpoint="j1-grande",
                                   prompt=your_prompt,
                                   maxTokens=100,
                                   temperature=0.7,
                                   stopSequences=['##'],
                                   numResults=3)

for comp in response['completions']:
    print(comp['data']['text'].strip())
    print("=============")

### Interested in learning more?
Take a look at our [blog post](https://www.ai21.com/blog/building-cv-profile-generator-using-ai21-studio) to understand the process of building a good prompt.

### F. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Clean-up

### A. Delete the model

In [ ]:
model.delete_model()